In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

# 02_开始使用 Keras 函数式 API

Keras 函数式 API 是定义复杂模型（如多输出模型、有向无环图，或具有共享层的模型）的方法。

相较上一篇笔记，这里着重解释API的详细用法。

## 一切皆为张量：一个简单的例子--全连接网络

这个例子能够帮助我们对Keras API进行一些简单的理解。对于全连接网络，Sequential 模型可能是实现这种网络的一个更好选择。需要注意：

- 网络层的实例是可调用的，它以张量为参数，并且返回一个张量
- 输入和输出均为张量，它们都可以用来定义一个模型（Model）
- 这样的模型同 Keras 的 Sequential 模型一样，都可以被训练

In [ ]:
# 这部分返回一个张量
inputs = Input(shape=(784,))

# 层的实例是可调用的，它以张量为参数，并且返回一个张量
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# 这部分创建了一个包含输入层和三个全连接层的模型
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels)  # 开始训练

注意这里的一个写法：x = Dense(64, activation='relu')(inputs)

在 Layer 这个类里面，有这样一个函数(https://github.com/keras-team/keras/blob/4cde148de0c37981c50f3a8e4a59fa4e5f653e17/keras/engine/topology.py#L541)：

def __call__(self, inputs, **kwargs):

通过定义这个函数，我们可以让一个对象可以当函数用。因此我们可以把下面的代码拆成这样：

在Python的官方文档中有详细说明(https://docs.python.org/3/reference/datamodel.html#emulating-callable-objects)

3.3.6. Emulating callable objects

object.__call__(self[, args...])

    Called when the instance is “called” as a function; if this method is defined, x(arg1, arg2, ...) is a shorthand for x.__call__(arg1, arg2, ...).